In [1]:
import torch
GPU_name = torch.cuda.get_device_name()
print(GPU_name)
if GPU_name == "Tesla V100-SXM2-16GB":
    print("Get V100!")
    print("●●●●●")

Tesla V100-SXM2-16GB
Get V100!
●●●●●


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2021spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1aQoWogAQo_xVJvMQMrGaYiWzuyfO0QyLLAhiMwFyS2w)　Kaggle: [Link](https://www.kaggle.com/c/ml2021-spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Download Dataset

In [3]:
# Download link 1
!gdown --id '1kBW411aaw1zFGox-OvVXmj3OexCoRd45' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1pOu3FdPdvzielUZyggeD7KDnVy9iW1uC' --output hw7_data.zip

!unzip -o hw7_data.zip

Downloading...
From: https://drive.google.com/uc?id=1kBW411aaw1zFGox-OvVXmj3OexCoRd45
To: /content/hw7_data.zip
7.71MB [00:00, 21.2MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          


## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [1]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

## Import Packages

In [1]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [2]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True # False

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

## Load Model and Tokenizer




 

other model<br/>
https://huggingface.co/models

In [3]:
# model = BertForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

# model = BertForQuestionAnswering.from_pretrained("hfl/chinese-macbert-base").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-macbert-base")

model = BertForQuestionAnswering.from_pretrained("hfl/chinese-roberta-wwm-ext-large").to(device)
tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-roberta-wwm-ext-large")

# model = BertForQuestionAnswering.from_pretrained("hfl/chinese-roberta-wwm-ext").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-roberta-wwm-ext")

# model = BertForQuestionAnswering.from_pretrained("hfl/rbt4").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("hfl/rbt4")

# model = BertForQuestionAnswering.from_pretrained("hfl/rbtl3").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("hfl/rbtl3")

# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("Geotrend/bert-base-zh-cased")
# model = AutoModel.from_pretrained("Geotrend/bert-base-zh-cased")

# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("amine/bert-base-5lang-cased")
# model = AutoModel.from_pretrained("amine/bert-base-5lang-cased")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext-large were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl

## Read Data

- Training set: 26935 QA pairs
- Dev set: 3523  QA pairs
- Test set: 3492  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [4]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [5]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

In [6]:
print(type(dev_paragraphs))
print(len(dev_paragraphs[0]))
print(dev_paragraphs[0])
print(type(dev_paragraphs[0]))
print(type(len(dev_paragraphs)))
print(len(dev_paragraphs))

<class 'list'>
334
在歐洲，梵語的學術研究，由德國學者陸特和漢斯雷頓開創。後來威廉·瓊斯發現印歐語系，也要歸功於對梵語的研究。此外，梵語研究，也對西方文字學及歷史語言學的發展，貢獻不少。1786年2月2日，亞洲協會在加爾各答舉行。會中，威廉·瓊斯發表了下面這段著名的言論：「梵語儘管非常古老，構造卻精妙絕倫：比希臘語還完美，比拉丁語還豐富，精緻之處同時勝過此兩者，但在動詞詞根和語法形式上，又跟此兩者無比相似，不可能是巧合的結果。這三種語言太相似了，使任何同時稽考三者的語文學家都不得不相信三者同出一源，出自一種可能已經消逝的語言。基於相似的原因，儘管缺少同樣有力的證據，我們可以推想哥德語和凱爾特語，雖然混入了迥然不同的語彙，也與梵語有著相同的起源；而古波斯語可能也是這一語系的子裔。」
<class 'str'>
<class 'int'>
1000


In [7]:
tokenizer.convert_tokens_to_ids(['[UNK]', 'Hello', 'word', '!', '[SEP]'])

[100, 100, 8681, 106, 102]

## Dataset and Dataloader

In [8]:
import random
devtrainflag = False
from torch.utils.data import ConcatDataset

class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 200 #window size 150
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 40 # 150

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn
        # make answer appear in different place in a single window
        
        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer

            mid = (answer_start_token + answer_end_token) // 2
            
            a = random.randint(-self.max_paragraph_len// 2, self.max_paragraph_len// 2)
            # print("cool")
            # paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_start = max(0, min(mid + a - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))

            paragraph_end = paragraph_start + self.max_paragraph_len
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        if self.split == "devtrain":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer

            mid = (answer_start_token + answer_end_token) // 2
            
            a = random.randint(-self.max_paragraph_len// 2, self.max_paragraph_len// 2)
            # print("cool")
            # paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_start = max(0, min(mid + a - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))

            paragraph_end = paragraph_start + self.max_paragraph_len
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
traindev_set = QA_Dataset("devtrain", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
train_setall = ConcatDataset([train_set, traindev_set])


dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)



train_batch_size = 16 # 16

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
if devtrainflag:
  train_loader = DataLoader(train_setall, batch_size=train_batch_size, shuffle=True, pin_memory=True)
else:
  train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)

dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [10]:
import re

def evaluate(data, output,evaset, id):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # print(start_prob)
            # print(start_index)
            # print(end_prob)
            # print(end_index)

            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            if end_index < start_index :
              continue

            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])

            

    # Remove spaces in answer (e.g. "大 金" --> "大金")
    answer = answer.replace(' ','')
    # answer = answer.replace('《','')
    # answer = answer.replace('》','')
    answer = answer.replace('—','')



    if answer.find("[UNK]") != -1:
      print(answer)
      answer = answer.replace('[UNK]', '.')
      print(answer)
      if evaset == 'dev':
        paragraphid = dev_questions[id]["paragraph_id"]
        f = re.search(answer, dev_paragraphs[paragraphid])
        if f != None:
          # print(f.start(), f.end())
          # print( f.group(0) )
          answer = f.group(0)
          print(answer)
      if evaset == 'test':
        paragraphid = test_questions[id]["paragraph_id"]
        # print(id)
        # print(paragraphid)
        f = re.search(answer, test_paragraphs[paragraphid])
        if f != None:
          # print(f.start(), f.end())
          # print( f.group(0) )
          answer = f.group(0)
          print(answer)
        else:
          print('not found')   

    return answer

## Training & Testing

In [11]:
# may load model here
# model = BertForQuestionAnswering.from_pretrained("saved_model")
lrlastlog = 0
num_epoch = 1 #1
validation = True # dont want to use vali set train -> set to False
logging_step = 100
learning_rate = 3e-5 # 1e-4 (2e-5-4e-5)
# total_steps = 1684 * 16 * num_epoch / train_batch_size
if devtrainflag:
  total_steps = 1904
else:
  total_steps = 1684

optimizer = AdamW(model.parameters(), lr=learning_rate)
bestacc = 0
if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

model.train()

# optimizer.param_groups[0]['lr'] = 0


print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0

    if epoch == 1:
      optimizer.param_groups[0]['lr'] = lrlastlog
    if epoch == 2:
      optimizer.param_groups[0]['lr'] = lrlastlog  
    for data in tqdm(train_loader):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])
        # output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss
        
        if fp16_training:
            accelerator.backward(output.loss)
        else:
            output.loss.backward()
        

        
        optimizer.step()
        optimizer.zero_grad()
        step += 1

        ##### TODO: Apply linear learning rate decay + warmup#####
        # learning rate decay
        if epoch == 0:
          optimizer.param_groups[0]['lr'] =  optimizer.param_groups[0]['lr'] - (learning_rate / total_steps)
        # learning rate decay + warmup
        # warmupsteps = 100
        # if step <= warmupsteps :
        #   optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] + (learning_rate / warmupsteps)
        # else:  
        #   optimizer.param_groups[0]['lr'] =  optimizer.param_groups[0]['lr'] - (learning_rate / (total_steps-warmupsteps))

        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            print("lr = " + str(optimizer.param_groups[0]['lr']))
            lrlastlog = optimizer.param_groups[0]['lr']
            train_loss = train_acc = 0



if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device), attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output, evaset='dev', id = i) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        if ( (dev_acc / len(dev_loader)) > bestacc):
          bestacc = (dev_acc / len(dev_loader))
          
          # Save a model and its configuration file to the directory 「saved_model」 
          # i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
          # Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
          print("Saving Model ...")
          model_save_dir = "saved_model"
          gmodel_save_dir = "/content/drive/MyDrive/ML/HW7/saved_model"
          model.save_pretrained(model_save_dir)
          model.save_pretrained(gmodel_save_dir)


        model.train()



print("Evaluating Test Set ...")

result = []

model = BertForQuestionAnswering.from_pretrained("saved_model").to(device)

model.eval()
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output, evaset='test', id = i))

result_file = "result.csv"
gresult_file = "/content/drive/MyDrive/ML/HW7/result.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

with open(gresult_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {gresult_file}")


Start Training ...


Epoch 1 | Step 100 | loss = 1.718, acc = 0.483
lr = 2.823634204275527e-05
Epoch 1 | Step 200 | loss = 0.784, acc = 0.691
lr = 2.6454869358669683e-05
Epoch 1 | Step 300 | loss = 0.688, acc = 0.737
lr = 2.4673396674584096e-05
Epoch 1 | Step 400 | loss = 0.584, acc = 0.761
lr = 2.289192399049851e-05
Epoch 1 | Step 500 | loss = 0.586, acc = 0.761
lr = 2.1110451306412922e-05
Epoch 1 | Step 600 | loss = 0.555, acc = 0.777
lr = 1.9328978622327335e-05
Epoch 1 | Step 700 | loss = 0.556, acc = 0.772
lr = 1.7547505938241748e-05
Epoch 1 | Step 800 | loss = 0.532, acc = 0.796
lr = 1.576603325415616e-05
Epoch 1 | Step 900 | loss = 0.493, acc = 0.782
lr = 1.3984560570070574e-05
Epoch 1 | Step 1000 | loss = 0.485, acc = 0.794
lr = 1.2203087885984987e-05
Epoch 1 | Step 1100 | loss = 0.509, acc = 0.788
lr = 1.04216152018994e-05
Epoch 1 | Step 1200 | loss = 0.473, acc = 0.817
lr = 8.640142517813813e-06
Epoch 1 | Step 1300 | loss = 0.489, acc = 0.785
lr = 6.858669833728263e-06
Epoch 1 | Step 1400 | loss =

大肚平埔族拍布拉族大肚王與瑯[UNK]番人的反抗
大肚平埔族拍布拉族大肚王與瑯.番人的反抗
大肚平埔族拍布拉族大肚王與瑯嶠番人的反抗
《阿[UNK]婆吠陀》
《阿.婆吠陀》
《阿闥婆吠陀》
姚[UNK]
姚.
姚萇
[UNK]水之戰
.水之戰
淝水之戰
美國長[UNK]米
美國長.米
美國長秈米
測試[UNK][UNK]村附近男性居民的脫氧核糖核酸
測試..村附近男性居民的脫氧核糖核酸
測試驪靬村附近男性居民的脫氧核糖核酸
金[UNK]
金.
金入
奕[UNK]
奕.
奕訢
[UNK]族
.族
撣族
3000[UNK]顆
3000.顆
3000垓顆
回[UNK]汗國
回.汗國
回鶻汗國
白[UNK]紀
白.紀
白堊紀
白[UNK]紀中期
白.紀中期
白堊紀中期
回[UNK]部落
回.部落
回鶻部落
白[UNK]紀
白.紀
白堊紀
蔡[UNK]
蔡.
蔡鍔
久彌宮妃[UNK]子
久彌宮妃.子
久彌宮妃俔子
船[UNK]·庭院
船.·庭院
船圬·庭院
《阿[UNK]婆吠陀》
《阿.婆吠陀》
《阿闥婆吠陀》
其中含有[UNK]
其中含有.
其中含有烴
[UNK]靼
.靼
韃靼
無脊椎動物[UNK]如昆蟲
無脊椎動物.如昆蟲
無脊椎動物—如昆蟲
盤[UNK]蠻
盤.蠻
盤瓠蠻
李[UNK]
李.
李世
朱允[UNK]
朱允.
朱允炆

Validation | Epoch 1 | acc = 0.821
Saving Model ...
Evaluating Test Set ...


溥[UNK]
溥.
溥儁
目前沒有觀察到任何語言純[UNK]以力道來區分不同輔音
目前沒有觀察到任何語言純.以力道來區分不同輔音
目前沒有觀察到任何語言純綷以力道來區分不同輔音
[UNK]人國
.人國
荇人國
[UNK][UNK]
..
高句
馬[UNK]
馬.
馬馼
東晉常[UNK]
東晉常.
東晉常璩
[UNK]稻
.稻
對稻
白[UNK]紀滅絕事件
白.紀滅絕事件
白堊紀滅絕事件
抗佝[UNK]病維他命
抗佝.病維他命
抗佝僂病維他命
杭州[UNK]橋機場
杭州.橋機場
杭州筧橋機場
蔡[UNK]
蔡.
蔡鍔
丁[UNK]
丁.
丁旿
隋[UNK]帝
隋.帝
隋煬帝
胡季[UNK]
胡季.
胡季犛
其英文縮寫首字母為「[UNK]·ㄎㄟ·ㄨㄞ」
其英文縮寫首字母為「.·ㄎㄟ·ㄨㄞ」
not found
梁[UNK]
梁.
梁鵠
[UNK]靼海峽
.靼海峽
韃靼海峽
侏[UNK]紀
侏.紀
侏儸紀
克里米亞[UNK]靼人
克里米亞.靼人
克里米亞韃靼人
白[UNK]紀中期
白.紀中期
白堊紀中期

Completed! Result is in result.csv
Completed! Result is in /content/drive/MyDrive/ML/HW7/result.csv
